In [ ]:
# You can omit this cell if you already have pkl files
from feature_extraction import Method
from readers import GetProcessedInputData
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy

feature_extraction_methods = \
  [Method.BLACK_PIXEL_RATIO, Method.BLACK_PIXEL_RATIO_LC_UP, Method.LOWER_CONTOUR, Method.UPPER_CONTOUR]
transcriptions_as_list, keywords_to_search, train_words, test_words = GetProcessedInputData(feature_extraction_methods)

#Run line below if you would like to create dumpfiles
#dumpToFile(inputDataVariableNames) 

In [64]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import pandas as pd
inputDataVariableNames = ["transcriptions_as_list", "keywords_to_search", "train_words", "test_words"]
def dumpToFile(names):
    if isinstance(names, str):
        names= [names]
    for name in names:
        dumpFile = os.fsencode(os.getcwd() + "/output/" + name + ".pkl")
        with open(dumpFile,"wb") as f:
            pickle.dump(eval(name), f)
def dumpFromFile(names):
    if isinstance(names, str):
        names= [names]
    for name in names:
        print(name)
        dumpedFile = os.fsencode(os.getcwd() + "/output/" + name + ".pkl")
        with open(dumpedFile,"rb") as f:
            globals()[name] = pickle.load(f)
            
def pickleToCsv(filenames):
    if isinstance(filenames, str):
        filenames= [filenames]
    for name in filenames:
        with open(os.fsencode(os.getcwd() + "/output/" + name + ".pkl"), "rb") as f:
            l = pickle.load(f)
            df = pd.DataFrame(l)
            df.to_csv('output/{0}.csv'.format(name))

In [65]:
dumpFromFile(inputDataVariableNames)
pickleToCsv(inputDataVariableNames)

transcriptions_as_list
keywords_to_search
train_words
test_words


In [66]:
# TODO: For testing only. Remove the entire cell later

# all_train_words = train_words
# all_test_words = test_words

train_words = train_words[:20]
test_words = test_words[:20]

In [ ]:
import numpy as np
from dtaidistance import dtw
from compute_distances import ComputeDistances
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean


distances = ComputeDistances(train_words, test_words)

In [ ]:
import pickle   
#Put distances dictionary into a pickle file
#Put the name of you feature
file = open("output/FOUR_FEATURES.pkl", "wb")
pickle.dump(distances, file)
file.close()

In [ ]:
import pickle   
#Retrieve the distance dictionary from a pickle file
#Put the name of you feature
file_to_read = open("output/FOUR_FEATURES.pkl", "rb")
distances_load = pickle.load(file_to_read)
file_to_read.close()

In [60]:
print(distances_load)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from keyword_spotting import FindClosestKnownWord, SpotKeyword


# For each test_word, find the train_word that is closest to it.
for test_word in test_words:
  test_word.closest_train_word, test_word.distance_to_closest_train_word = \
    FindClosestKnownWord(test_word, train_words, distances)
    

# Search keywords and save spotted keywords in the list.
outputData = []
for keyword_to_search in keywords_to_search:
  spotted_test_keywords = SpotKeyword(keyword_to_search, test_words)
  if len(spotted_test_keywords):
    print(keyword_to_search)
    print(spotted_test_keywords)
    spottedKeyword = []
    for keyword in spotted_test_keywords:
      spottedKeyword.append(keyword.id)
      spottedKeyword.append(keyword.distance_to_closest_train_word)
      print(keyword.distance_to_closest_train_word)
      keyword.image.show()
    outputData.append([keyword_to_search, *spottedKeyword])
    input()

In [62]:
# TODO: write output file kws.csv (slide 24)
import csv
with open("kws.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(outputData)

In [ ]:
# Evaluation Metrics - In progress, not finished yet (There might be errors in the logic)
# convert transcriptions to dict - easier and faster access values later
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy
groundTruth = dict()
for sub in transcriptions_as_list:
    groundTruth[sub[0]] = sub[1]

testWords = dict()
for word in test_words:
    testWords[word.id] = word.transcription

predictions = dict()
for item in outputData:
    predictions[item[0]] = item[1::2]

acc = CalculateAccuracy(keywords_to_search, testWords, predictions, groundTruth)


print(acc.CalculateRecall())
print(acc.CalculatePrecision())
